# Autoencoder v2
Re-implemented v1 with enhancements

### Information
1. Eight-layer architecture (4 in encoder + 4 in decoder)
2. Dataset - 1000 STL files
3. Train - 800 models
4. Test - 200 models
5. Epochs - 50
6. Loss - 0.0214

### Encoder architecture
- Conv3D(32, (3, 3, 3), activation='elu', padding='same')
- MaxPooling3D((2, 2, 2), padding='same')
- Conv3D(16, (3, 3, 3), activation='elu', padding='same')
- MaxPooling3D((2, 2, 2), padding='same')

### Changelog
#### First attempt
- Modified ConversionUtils.py so that the point clouds generated have 15k points instead of 10k
- Result: Loss reduced from 0.0266 to 0.0230
#### Second attempt
- Switched from ReLU to ELU
- Result: Loss reduced from 0.0230 to 0.0216
#### Third attempt
- Modified ConversionUtils.py so that the point clouds generated have 20k points instead of 15k
- Result: Loss reduced from 0.0216 to 0.0214

### Importing all necessary libraries

In [ ]:
import numpy as np
import tensorflow as tf
from keras.layers import Input, Conv3D, MaxPooling3D, UpSampling3D, GlobalMaxPooling3D
from keras.models import Model
import os
import numpy as np
import pandas as pd
from pyntcloud import PyntCloud
import open3d as o3d
from ConversionUtils import ConversionUtils
from Visualization import Visualization

### Converting STL to point cloud

**Execute the following cell if you wish to delete all existing point cloud files in abc-dataset-ply/ directory**

In [ ]:
folder_path = "abc-dataset-ply/"
files = os.listdir(folder_path)

for file in files:
    file_path = os.path.join(folder_path, file)
    if os.path.isfile(file_path):
        try:
            os.remove(file_path)
        except Exception as e:
            print(f"Error deleting {file}: {str(e)}")

**Only execute the next cell if the point cloud files do not exist in abc-dataset-ply/ directory**

In [ ]:
files = sorted(ConversionUtils.list_files_in_directory("abc-dataset-stl/"))
for i in files:
    path = "abc-dataset-stl/" + i
    ConversionUtils.stl_to_ply(path, 20000)

### Converting point cloud to binary voxel arrays

In [ ]:
directory = "abc-dataset-ply/"
# Taking first 1000 models
files = sorted([filename for filename in os.listdir(directory) if os.path.isfile(os.path.join(directory, filename))])
dataset = []

for i in files:
    path = os.path.join(directory, i)
    binvox = ConversionUtils.convert_to_binvox(path, 64)
    dataset.append(binvox)

In [ ]:
print(len(dataset))

### Splitting dataset for training and testing

In [ ]:
# Load your dataset into numpy arrays
dataset = np.array(dataset)

# Split your dataset into train and test datasets
train_dataset = dataset[:800]  # Adjust the number as needed
test_dataset = dataset[800:]   # The remaining data for testing
print(len(train_dataset), len(test_dataset))
# Define the input shape
input_shape = (64, 64, 64, 1)

### Encoder

In [ ]:
input_data = Input(shape=input_shape)
x = Conv3D(32, (3, 3, 3), activation='elu', padding='same')(input_data)
x = MaxPooling3D((2, 2, 2), padding='same')(x)
x = Conv3D(16, (3, 3, 3), activation='elu', padding='same')(x)
encoded = MaxPooling3D((2, 2, 2), padding='same')(x)

### Decoder

In [ ]:
x = Conv3D(16, (3, 3, 3), activation='elu', padding='same')(encoded)
x = UpSampling3D((2, 2, 2))(x)
x = Conv3D(32, (3, 3, 3), activation='elu', padding='same')(x)
x = UpSampling3D((2, 2, 2))(x)
decoded = Conv3D(1, (3, 3, 3), activation='sigmoid', padding='same')(x)

### Training autoencoder, prediction done on test dataset

In [ ]:
autoencoder = Model(input_data, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

autoencoder.fit(train_dataset, train_dataset, epochs=50, batch_size=10, validation_data=(test_dataset, test_dataset))

In [ ]:
reconstructed_data = autoencoder.predict(test_dataset, batch_size=10)

In [ ]:
encoder = Model(inputs=input_data, outputs=encoded)
encoded_data = encoder.predict(test_dataset, batch_size=10)

#### Dimensions

In [ ]:
print("Shape of input data: ", test_dataset[0].shape)
print("Shape of encoded data: ", encoded_data[0].shape)
print("Shape of reconstructed data: ", reconstructed_data[0].shape)

In [ ]:
# This will convert the encoded data of each model (which is of shape 16x16x16x16) to a 1D array of 65536 elements
encoded_data_flattened = encoded_data[0].flatten()
print("Shape of encoded data after flattening: ", encoded_data_flattened.shape)
# This will convert the above array of 65536 elements back to the original encoder output of 8x8x8x8 dimensions (4D array)
encoded_regenerated = encoded_data_flattened.reshape(16, 16, 16, 16)
print("Shape of encoded data after reshaping flattened array format: ", encoded_regenerated.shape)

### Sample reconstruction from test dataset

In [ ]:
import matplotlib.pyplot as plt

index = 0

original_sample = test_dataset[index]

reconstructed_sample = reconstructed_data[index].reshape(64, 64, 64)
threshold = 0.35
reconstructed_sample = (reconstructed_sample > threshold).astype(int)

In [ ]:
Visualization.matplotlib_visualize_original(original_sample)

In [ ]:
Visualization.matplotlib_visualize_reconstructed(reconstructed_sample)

In [ ]:
Visualization.open3d_visualize_original(original_sample)

In [ ]:
Visualization.open3d_visualize_reconstructed(reconstructed_sample)

In [ ]:
path = "sample-outputs/v2/" + "original-" + str(index) + ".ply"
o3d.io.write_point_cloud(path, original_sample)

In [ ]:
path = "sample-outputs/v2/" + "reconstructed-" + str(index) + ".ply"
o3d.io.write_point_cloud(path, reconstructed_sample)